# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix (q=0,ω=0) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234919635411                   -0.50    7.0
  2   -7.250257701156       -1.81       -1.40    1.0
  3   -7.251233550375       -3.01       -2.03    4.0
  4   -7.251023618433   +   -3.68       -1.92    4.0
  5   -7.251336014618       -3.51       -2.91    3.0
  6   -7.251338673200       -5.58       -3.50    3.0
  7   -7.251338793197       -6.92       -4.15    2.0
  8   -7.251338797534       -8.36       -4.51    3.0
  9   -7.251338798650       -8.95       -5.11    2.0
 10   -7.251338798700      -10.30       -5.68    3.0
 11   -7.251338798704      -11.41       -6.40    3.0
 12   -7.251338798705      -12.79       -6.77    3.0
 13   -7.251338798705      -13.55       -7.33    3.0
 14   -7.251338798705      -14.75       -7.89    2.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Apply ε† = 1 - χ0 (vc + fxc)

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end

eps_fun (generic function with 1 method)

eager diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.06535180280271975
[ Info: Arnoldi iteration step 2: normres = 0.5107852914916206
[ Info: Arnoldi iteration step 3: normres = 0.8446729035665133
[ Info: Arnoldi iteration step 4: normres = 0.3497353969578608
[ Info: Arnoldi iteration step 5: normres = 0.2801117498865412
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (3.45e-02, 3.34e-02, 1.61e-01, 2.23e-01, 2.11e-02)
[ Info: Arnoldi iteration step 6: normres = 0.4810184659112314
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.25e-02, 4.06e-02, 4.56e-01, 8.82e-02, 8.41e-02)
[ Info: Arnoldi iteration step 7: normres = 0.0846056795870268
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (5.99e-04, 7.35e-03, 3.83e-02, 3.28e-02, 6.13e-02)
[ Info: Arnoldi iteration step 8: normres = 0.12672914385649456
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (3.27e-05